In [6]:
from llama_index.legacy import (
    KeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    download_loader,
    load_index_from_storage,
    VectorStoreIndex)

from llama_index.legacy.llms.vertex import Vertex
from langchain.chat_models import ChatVertexAI
from llama_index.legacy.llms.langchain import LangChainLLM
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.legacy.embeddings import LangchainEmbedding
from llama_index.legacy import Document
from llama_index.legacy.node_parser import SimpleNodeParser, HierarchicalNodeParser


from llama_index.legacy.vector_stores.chroma import ChromaVectorStore
import chromadb


from google.oauth2 import service_account
from llama_index.legacy import set_global_service_context
from google.cloud import aiplatform
import json 

## imports from og llama index file

In [7]:
from llama_index.legacy import VectorStoreIndex, SimpleDirectoryReader
from llama_index.legacy.embeddings import LangchainEmbedding
from llama_index.legacy.text_splitter import TokenTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.legacy.llms.vertex import Vertex
from llama_index.legacy.node_parser import SimpleNodeParser, HierarchicalNodeParser
from llama_index.legacy import (
    KeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    download_loader,
    load_index_from_storage,
    VectorStoreIndex)
from llama_index.legacy.retrievers import VectorIndexRetriever
from llama_index.legacy.prompts import (
    ChatMessage,
    ChatPromptTemplate,
    MessageRole,
    PromptTemplate,
)

from llama_index.legacy.postprocessor import NERPIINodePostprocessor, SentenceEmbeddingOptimizer
from llama_index.legacy import ServiceContext
from llama_index.legacy.schema import QueryBundle
from llama_index.legacy.schema import NodeWithScore, TextNode

from pathlib import Path
from tqdm.notebook import tqdm
from google.oauth2 import service_account

from llama_index.legacy import set_global_service_context
import re
import uuid
import os
from pathlib import Path
from pprint import pprint
import pandas as pd
import csv
from typing import List, Tuple, Dict
import time
import json

from llama_index.legacy.vector_stores.chroma import ChromaVectorStore
from llama_index.legacy import StorageContext
from llama_index.legacy.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
from llama_index.core import Settings

from llama_index.legacy.llms.langchain import LangChainLLM
from langchain_google_vertexai import ChatVertexAI
from llama_index.legacy import Response
from llama_index.legacy.response_synthesizers import Refine
from llama_index.legacy.evaluation import SemanticSimilarityEvaluator
from llama_index.legacy.evaluation import RelevancyEvaluator
from llama_index.legacy.embeddings import SimilarityMode

import nest_asyncio
nest_asyncio.apply()

# additional
from llama_index.legacy.postprocessor import LongContextReorder
from llama_index.legacy.retrievers import AutoMergingRetriever
from llama_index.legacy.query_engine import TransformQueryEngine
from llama_index.legacy.query_engine import RetrieverQueryEngine
from llama_index.legacy.indices.query.query_transform.base import (
    HyDEQueryTransform,StepDecomposeQueryTransform
)
from llama_index.legacy.node_parser import get_leaf_nodes
from llama_index.legacy import Document
# llama_debug = LlamaDebugHandler(print_trace_on_end=True)
# callback_manager = CallbackManager([llama_debug])

In [8]:
## Initialize credentials, models, and embedding models

In [9]:
credentials = service_account.Credentials.from_service_account_file("credentials/vertex-test-417403-ce72ad032af7.json")
vertex_ai = Vertex(model="text-bison", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2)
chat_vertex_ai = ChatVertexAI(model_name="chat-bison-32k", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2, max_output_tokens= 8192) # max for bison 32k
embed_model = LangchainEmbedding(VertexAIEmbeddings(model_name='textembedding-gecko-multilingual@latest'))

In [10]:
service_context = ServiceContext.from_defaults(llm=vertex_ai, embed_model=embed_model, chunk_size=1024, chunk_overlap=20)
set_global_service_context(service_context)
# need to set global service context becuase if not index = VectorStoreIndex.from_vector_store(vector_store, embed_model=Settings.embed_model) will produce an error
# it will say that it requires OPENAI key etc
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=3900, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='textembedding-gecko-multilingual@latest', embed_batch_size=10, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x000001CCBB0A6710>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x000001CCBB0A6710>, id_func=<function default_id_func at 0x000001CCAB630F40>, chunk_size=1024, chunk_overlap=20, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.legacy.logger.base.LlamaLogger object at 0x000001CCBBCFC450>, callback_manager=<llama_index.legacy.callbacks.base.

## Import and load data
- How to create documents from a list of strings: 
https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents.html#defining-documents

### load csv data

cleaning csv data

In [11]:
import pandas as pd

# unused
# reader = SimpleDirectoryReader(
#     input_files=["../data/siriraj_doctor_details.csv"]
# )
# csv_data = reader.load_data()
# print(f"Loaded {len(csv_data)} docs")
# csv_data

csv_file_path = "../data/siriraj_doctor_details.csv"
csv_df = pd.read_csv(csv_file_path)
columns_to_drop = ['image_src', 'url']
df = csv_df.drop(columns=[col for col in csv_df.columns if col in columns_to_drop])
df

,name,table_check,qualification,expertise
0,รศ. พญ.กติกา นวพันธุ์,"เสาร์ 14:00 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5",แพทยศาสตรบัณฑิต คณะแพทยศาสตร์ จุฬาลงกรณ์มหาวิท...,เวชศาสตร์มารดาและทารกในครรภ์
1,รศ. พญ.กนกรัตน์ พรพาณิชย์,"พุธ 08:00 - 11:15 น. สัปดาห์ที่ 1,2,3,4,5",การถนอมสายตา,ศัลยกรรมกระดูกเบ้าตาและตกแต่ง
2,อ. พญ.กนกรัตน์ สุวรรณสิทธิ์,"พฤหัสบดี 13:30 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5",แพทยศาสตรบัณฑิต คณะแพทยศาสตร์ศิริราชพยาบาล มหา...,โสต ประสาท การได้ยินและการทรงตัว
3,ทพญ.กนกอร ตันติพาณิชย์กูล,"พุธ 16:30 - 19:30 น. สัปดาห์ที่ 3,4",ทันตแพทยศาสตร์บัณฑิต คณะทันตแพทยศาสตร์ มหาวิทย...,ทันตกรรมจัดฟัน การจัดฟันด้วยการผ่าตัดจัดฟัน
4,ผศ. นพ.กมล เผือกเพ็ชร,"พุธ 16:00 - 18:00 น. สัปดาห์ที่ 1,3,5 อาทิตย์ ...",โรคมือ เท้า ปาก(hand-foot-and-mouth disease),กุมารเวชศาสตร์โลหิตวิทยาและมะเร็งในเด็ก
...,...,...,...,...
625,นพ.เอกราช ลิ่วธนมงคล,"เสาร์ 09:00 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 13...",แพทยศาสตรบัณฑิต คณะแพทยศาสตร์ศิริราชพยาบาล มหา...,"โรคของลำไส้ใหญ่และทวารหนัก, ฝีคัณฑสูตร, ริดสีด..."
626,รศ. นพ.เอกรินทร์ โชติกวาณิชย์,"พุธ 13:00 - 13:30 น. สัปดาห์ที่ 1,2,3,4,5 เสาร...",ปัสสาวะบ่อย ปัสสาวะขัดกระเพาะปัสสาวะอักเสบหรือ...,"โรคนิ่วไต ท่อไต และกระเพาะปัสสาวะ, โรคเนื้องอก..."
627,อ. นพ.เอกวิทย์ เกยุราพันธุ์,"ศุกร์ 09:00 - 11:00 น. สัปดาห์ที่ 1,2,3,4,5 อา...",NaN,ศัลยศาสตร์ออร์โธปิดิกส์
628,นพ.เอนกชัย ดกพฤกษ์,NaN,เฝ้าระวังศีรษะกระแทก 72 ชั่วโมง,เวชศาสตร์ฉุกเฉิน


In [12]:
csv_data = df.apply(lambda x: ''.join(x.astype(str)), axis=1).tolist()
csv_data = [Document(text=t) for t in csv_data]
print(len(csv_data))
print(csv_data)

630
[Document(id_='5bba1b20-54ab-459a-81c7-3bf15dadc951', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='รศ. พญ.กติกา นวพันธุ์เสาร์ 14:00 - 17:00 น. สัปดาห์ที่ 1,2,3,4,5แพทยศาสตรบัณฑิต คณะแพทยศาสตร์ จุฬาลงกรณ์มหาวิทยาลัย พ.ศ.2544วุฒิบัตรผู้เชี่ยวชาญเฉพาะทางสาขาสูติศาสตร์และนรีเวชวิทยา จุฬาลงกรณ์มหาวิทยาลัย พ.ศ.2550วุฒิบัตรผู้เชี่ยวชาญเฉพาะทางสาขาเวชศาสตร์มารดาและทารกในครรภ์ คณะแพทยศาสตร์ศิริราชพยาบาล มหาวิทยาลัยมหิดล พ.ศ.2552Certificate in Fetal Intervention, University Hospital, Leuven, KU Leuven, Belgium (Research Fellow)พ.ศ.2557เวชศาสตร์มารดาและทารกในครรภ์', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='90b8ed32-4185-4540-840b-d9b81490e27c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='รศ. พญ.กนกรัตน์ พรพาณิชย์พุธ 08:00 - 11:15 น.

### load json data

In [13]:
json_file = "baby_0_3.json"
with open(json_file, 'r') as f:
    json_data = json.load(f)
    
# print("the number of documents: " , len(json_data))
# for i in json_data:
#     print(i) # str

How to create documents from a list of strings: 
https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents.html#defining-documents

In [14]:
from llama_index.legacy import Document
json_file = "baby_0_3.json"
with open(json_file, 'r') as f:
    json_data = json.load(f)

# inspecting json data
# print("the number of documents: " , len(json_data))
# for i in json_data:
#     print(i) # str
    
docs = [Document(text=t) for t in json_data]
print(len(docs))
print(docs)

101
[Document(id_='572249cd-a441-4a41-a48e-b48271035f57', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='1 คู่มือสําหรับพ่อแม่ เพื่อเผยแพร่ความรู\uf70bด\uf70bานการดูแลและพัฒนาเด็ก วัยเด็กเล็ก 0-3 ป\uf702 ตอน', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='ff608aa9-12c2-4654-9e6b-dd7e5477e23e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='2 คู่มือสำหรับพ่อแม่ เพื่อเผยแพร่ควำมรู้ด้ำนกำรดูแลและพัฒนำเด็ก ตอน วัยเด็กเล็ก 0-3 ปี ทีมบรรณาธิการ พญ.นัยนา ณีศะนันท์ พญ.จริยา ทะรักษา รศ.นพ.วีระศักดิ์ ชลไชยะ คณะอนุกรรมการ Child Health Supervision รศ.พญ.ประสบศรี อึ้งถาวร พญ.วันดี นิงสานนท์ ศ.คลินิก พญ.วินัดดา ปิยะศิลป์ รศ.พญ.จันท์ฑิตา พฤกษานานนท์ รศ.นพ.อดิศักดิ์ ผลิตผลการพิมพ์ รศ.นพ.พงษ์ศักดิ์ น้อยพยัคฆ์ ผศ.(พิเศษ)นพ.เทอดพงศ์ เต็มภาคย์ ISBN จัดพิม

In [15]:
docs.extend(csv_data)
print(len(docs))
print(docs)

731
[Document(id_='572249cd-a441-4a41-a48e-b48271035f57', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='1 คู่มือสําหรับพ่อแม่ เพื่อเผยแพร่ความรู\uf70bด\uf70bานการดูแลและพัฒนาเด็ก วัยเด็กเล็ก 0-3 ป\uf702 ตอน', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='ff608aa9-12c2-4654-9e6b-dd7e5477e23e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='2 คู่มือสำหรับพ่อแม่ เพื่อเผยแพร่ควำมรู้ด้ำนกำรดูแลและพัฒนำเด็ก ตอน วัยเด็กเล็ก 0-3 ปี ทีมบรรณาธิการ พญ.นัยนา ณีศะนันท์ พญ.จริยา ทะรักษา รศ.นพ.วีระศักดิ์ ชลไชยะ คณะอนุกรรมการ Child Health Supervision รศ.พญ.ประสบศรี อึ้งถาวร พญ.วันดี นิงสานนท์ ศ.คลินิก พญ.วินัดดา ปิยะศิลป์ รศ.พญ.จันท์ฑิตา พฤกษานานนท์ รศ.นพ.อดิศักดิ์ ผลิตผลการพิมพ์ รศ.นพ.พงษ์ศักดิ์ น้อยพยัคฆ์ ผศ.(พิเศษ)นพ.เทอดพงศ์ เต็มภาคย์ ISBN จัดพิม

### Create local ChromaDB Vectore Store (currently unused)

In [16]:
# # Create index and persist to Chroma vector store

# db = chromadb.PersistentClient(path="chroma_db")
# chroma_collection = db.get_or_create_collection("chroma_db")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

# index = VectorStoreIndex.from_documents(
#     documents = docs, storage_context=storage_context, embed_model=embed_model
# )

In [17]:
# Load index from persisted vector store

# currently index is used for retriever and not query engine
db = chromadb.PersistentClient(path="chroma_db")
chroma_collection = db.get_or_create_collection("chroma_db")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
chromadb_index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

chromadb_query_engine = chromadb_index.as_query_engine(similarity_top_k=2)
vector_retriever = chromadb_index.as_retriever(similarity_top_k=2)

### Configuring query engine components

Hierarchical Node Parsing

In [18]:
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[1024, 512, 256]
)
nodes = node_parser.get_nodes_from_documents(docs)

In [19]:
leaf_nodes = get_leaf_nodes(nodes)
nodes_by_id = {node.node_id: node for node in nodes}

In [20]:
auto_merging_context = ServiceContext.from_defaults(
    llm=LangChainLLM(ChatVertexAI(model_name="chat-bison@002",
                                         temperature=0.2,
                                         max_output_tokens=256)
                                         ),
    embed_model= LangchainEmbedding(VertexAIEmbeddings(model_name='textembedding-gecko-multilingual@latest')),
    node_parser=node_parser,
)

In [21]:
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

# leaf_nodes = get_leaf_nodes(nodes)
# nodes_by_id = {node.node_id: node for node in nodes}

# auto_merging_context = ServiceContext.from_defaults(
#     llm=selected_model,
#     embed_model= embed_model,
#     node_parser=node_parser,
# )

Store configured components in vector store called merging_index

In [22]:
# storage_context = StorageContext.from_defaults()
# storage_context.docstore.add_documents(nodes)

# automerging_index = VectorStoreIndex(leaf_nodes, storage_context=storage_context, service_context=auto_merging_context)
# # TODO: fix the code to store in chromadb
# # currently chromadb is not used

# merging_index_dir = "./merging_index"
# automerging_index.storage_context.persist(persist_dir=merging_index_dir)

Load automerging index

In [23]:
merging_index_dir = "./merging_index"
loaded_storage_context = StorageContext.from_defaults(persist_dir=merging_index_dir) # load the existing index
automerging_index = load_index_from_storage(loaded_storage_context)

In [24]:
automerging_as_retriever = automerging_index.as_retriever(similarity_top_k=10)
automerging_retriever = AutoMergingRetriever(
    automerging_as_retriever, 
    automerging_index.storage_context, 
    verbose=True
)

### Query engine

In [25]:
reorder = LongContextReorder()
hyde = HyDEQueryTransform(llm=vertex_ai, include_original=True)
retriever_query_engine = RetrieverQueryEngine.from_args(automerging_retriever,
                                              node_postprocessors=[reorder],
                                              )
transform_query_engine = TransformQueryEngine(retriever_query_engine, query_transform=hyde)

TESTING: Response production

In [26]:
query = "ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควรปรึกษาแพทย์ด้านใด"
response=transform_query_engine.query(query)
print("response: ", response.response)
# print("source nodes: ", response.source_nodes[0])
# print("source node text: ", response.source_nodes[0].node.text)
# print("number of source nodes: ", len(response.source_node))

> Merging 1 nodes into parent node.
> Parent node id: 27881fca-bc6b-48e9-bb74-599c8dc3e2f2.
> Parent node text: รศ. นพ.ชุณหเกษม   โชตินัยวัตรกุลอังคาร 09:00 - 12:00 น. สัปดาห์ที่ 1,2,3,4,5 13:00 - 16:00 น. สัป...

> Merging 1 nodes into parent node.
> Parent node id: 6cb6f748-779c-41f7-82a7-0da01c667512.
> Parent node text: รศ. นพ.เอกรินทร์ โชติกวาณิชย์พุธ 13:00 - 13:30 น. สัปดาห์ที่ 1,2,3,4,5 เสาร์ 09:00 - 12:00 น. สัป...

> Merging 1 nodes into parent node.
> Parent node id: 8158531f-e944-4480-af35-45cfe36d694e.
> Parent node text: รศ. นพ.สัชชนะ พุ่มพฤกษ์พุธ 13:00 - 14:30 น. สัปดาห์ที่ 1,2,3,4,5 อาทิตย์ 13:00 - 14:30 น. สัปดาห์...

> Merging 1 nodes into parent node.
> Parent node id: ea5df51b-34ef-476f-8d6b-b8c9b498ab87.
> Parent node text: อ. พญ.ชลัยรัชฎ์ สุขอวยชัยnanปัสสาวะบ่อย ปัสสาวะขัดกระเพาะปัสสาวะอักเสบหรือเปล่า?ศัลยศาสตร์ยูโร

> Merging 1 nodes into parent node.
> Parent node id: b6d718de-edae-40df-bc3f-0f7c7a906d11.
> Parent node text: รศ. นพ.ชุณหเกษม   โชตินัยวัตรกุลอังคาร

Inspecting response and context nodes

In [27]:
response.response

' ข้อมูลที่ให้มาไม่มีข้อมูลเกี่ยวกับอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกาย ดังนั้นจึงไม่สามารถตอบคำถามได้'

In [28]:
print(response.source_nodes[0])

Node ID: 31f3e0aa-13cf-4b2b-9fef-4abd757d8d69
Text: โรคบางอย่างท่ีนำมาด้วยอาการท้องผูก
เช่นโรคที่มีไทรอยด์ฮอร์โมนต�่า เด็กมักมีปัญหา พัฒนาการช้าร่วมด้วย ฯลฯ
หากสงสัยโรคดังกล่าวควรพาลูกไปพบแพทย์ วิธีการแก้ไข 1.
Score:  0.754



In [29]:
response.source_nodes[0].node.text

'โรคบางอย่างท่ีนำมาด้วยอาการท้องผูก เช่นโรคที่มีไทรอยด์ฮอร์โมนต�่า เด็กมักมีปัญหา พัฒนาการช้าร่วมด้วย ฯลฯ หากสงสัยโรคดังกล่าวควรพาลูกไปพบแพทย์ วิธีการแก้ไข 1.'

In [30]:
len(response.source_nodes)

10

Context Transformations - PII Example
- adding context to model response generation

In [31]:
pii_processor = NERPIINodePostprocessor()

def filter_pii_fn(**kwargs):
    # run optimizer
    query_bundle = QueryBundle(query_str=kwargs["query_str"])

    new_nodes = pii_processor.postprocess_nodes(
        [NodeWithScore(node=TextNode(text=kwargs["context_str"]))],
        query_bundle=query_bundle,
    )
    new_node = new_nodes[0]
    return new_node.get_content()

In [32]:
import llama_index.legacy.response_synthesizers

qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "If you need to state yes or no, state it in Thai only\n"
    "If the question is a yes/no question, answer either yes or no in Thai before giving your reasoning for the answer.\n"
    "The reasoning should contain the relevant information and be concise if possible. Don't reiterate the same information twice.\n"
    "If the question asks about the date and time for an apppointment answer in the following format only.\n"
    "(day) (time) (full name of doctor) (doctor's expertise)\n"
    "for example: วันจันทร์ เวลา 09:00 - 13:00 น. doctor: รศ. พญ.กติกา นวพันธุ์ expertise: เวชศาสตร์ฟื้นฟู \n" 
    "Also, the format of the answer should be as similar to the format in the context information as possible. This includes numbering order and indentation. \n"
    "The answer should not include the name of the document where the information is gotten from.\n"
    # "However, if the context information does not contain the answer for the query, answer เอกสารไม่มีระบุไว้\n"
    "Query: {query_str}\n"
    "Answer: "
)

qa_prompt_tmpl = PromptTemplate(
    qa_prompt_tmpl_str, function_mappings={"context_str": filter_pii_fn}
)


prompts_dict = transform_query_engine.get_prompts()
print(list(prompts_dict.keys()))

transform_query_engine.update_prompts(
    {"query_engine:response_synthesizers": qa_prompt_tmpl}
) # use merging index


['query_transform:hyde_prompt', 'query_engine:response_synthesizer:text_qa_template', 'query_engine:response_synthesizer:refine_template']


In [33]:
### Produce dataframe containing model response and context

In [34]:
def produce_response(query):
    # response = chromadb_query_engine.query(query)
    response = transform_query_engine.query(query)
    return response

def produce_context(query):
    # retrieved_nodes = vector_retriever.retrieve(query)
    retrieved_nodes = automerging_retriever.retrieve(query)
    context_str = "\n\n".join([n.get_content() for n in retrieved_nodes])
    return context_str

# model_resp_df['model_response'] = model_resp_df['question'].apply(produce_response)
# time.sleep(5)
# model_resp_df['context'] = model_resp_df['question'].apply(produce_context)

Testing response production

In [35]:
query = "ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควรปรึกษาแพทย์ด้านใด"
query = "ดิฉันต้องการนัดพบนักจิตวิทยาเพื่อการปรึกษาเรื่องความเครียด สะดวกมาพบเฉพาะสัปดาห์แรกและสัปดาห์สุดท้ายของเดือน สามารถมาหาหมอได้วันไหนบ้าง"
response = produce_response(query)
print("response:" , response)

> Merging 1 nodes into parent node.
> Parent node id: 139353a2-3eef-47fd-9fa9-a3af113f9dc7.
> Parent node text: รศ. นพ.สนทรรศ บุษราทิจจันทร์ 13:00 - 16:00 น. สัปดาห์ที่ 1,2,3,4,5 อังคาร 10:00 - 12:00 น. สัปดาห...

> Merging 1 nodes into parent node.
> Parent node id: c3e569fd-0ab8-4351-92bf-934d6ee122ea.
> Parent node text: รศ. พญ.พรจิรา ปริวัชรากุลพุธ 13:20 - 15:00 น. สัปดาห์ที่ 1,2,3,4,5 เสาร์ 13:00 - 14:20 น. สัปดาห์...

> Merging 1 nodes into parent node.
> Parent node id: 7d5b876d-644a-486c-aba3-f4294244d219.
> Parent node text: รศ. นพ.สุรศักดิ์ อังสุวัฒนาอังคาร 09:00 - 12:00 น. สัปดาห์ที่ 1,2,3,4 09:00 - 19:00 น. สัปดาห์ที่...

> Merging 1 nodes into parent node.
> Parent node id: 943e1b99-6c45-459c-80a8-9b69c518a5a3.
> Parent node text: ศ. ดร. นพ.นันทวัช สิทธิรักษ์พฤหัสบดี 16:30 - 20:00 น. สัปดาห์ที่ 1,2,3,4,5nanจิตเวชศาสตร์

> Merging 1 nodes into parent node.
> Parent node id: 9f56a3f9-e823-45ec-8a22-d91d600ce45b.
> Parent node text: รศ. นพ.สนทรรศ บุษราทิจจันทร์ 13:00 - 16:00 

In [36]:
context =  produce_context(query)
print("context: " , context)

> Merging 1 nodes into parent node.
> Parent node id: 7d5b876d-644a-486c-aba3-f4294244d219.
> Parent node text: รศ. นพ.สุรศักดิ์ อังสุวัฒนาอังคาร 09:00 - 12:00 น. สัปดาห์ที่ 1,2,3,4 09:00 - 19:00 น. สัปดาห์ที่...

> Merging 1 nodes into parent node.
> Parent node id: b6c4d4ec-88e6-45a7-9d08-d2093299c56d.
> Parent node text: รศ. นพ.มงคล เบญจาภิบาลอังคาร 09:30 - 12:00 น. สัปดาห์ที่ 1,2,3,4,5 พฤหัสบดี 09:30 - 12:30 น. สัปด...

> Merging 1 nodes into parent node.
> Parent node id: c3e569fd-0ab8-4351-92bf-934d6ee122ea.
> Parent node text: รศ. พญ.พรจิรา ปริวัชรากุลพุธ 13:20 - 15:00 น. สัปดาห์ที่ 1,2,3,4,5 เสาร์ 13:00 - 14:20 น. สัปดาห์...

> Merging 1 nodes into parent node.
> Parent node id: 139353a2-3eef-47fd-9fa9-a3af113f9dc7.
> Parent node text: รศ. นพ.สนทรรศ บุษราทิจจันทร์ 13:00 - 16:00 น. สัปดาห์ที่ 1,2,3,4,5 อังคาร 10:00 - 12:00 น. สัปดาห...

> Merging 1 nodes into parent node.
> Parent node id: 49e9d5cb-320f-4b18-9389-9d49a0620bdc.
> Parent node text: รศ. นพ.อัมพัน เฉลิมโชคเจริญกิจจั